<a href="https://colab.research.google.com/github/bhaweshiitk/20.C51_project/blob/main/Baseline_Models_20_C51_Graduate_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import matplotlib
import torch
from torch import nn
from torch.nn import functional as F
from sklearn.model_selection import train_test_split
!pip install Bio
!pip install umap-learn
from Bio import SeqIO
import umap

  Using cached bio-1.3.8-py3-none-any.whl (269 kB)
  Using cached mygene-3.2.2-py2.py3-none-any.whl (5.4 kB)
  Using cached biopython-1.79-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (2.3 MB)
  Using cached biothings_client-0.2.6-py2.py3-none-any.whl (37 kB)


In [5]:
# RNN, Hybrid RNN-CNN
# Linear regression, MLP, Random Forest
# Hyperparam tuning 
# Distance between train and test set

In [6]:
# Import data
! wget https://raw.githubusercontent.com/bhaweshiitk/3.C51_project/main/megares_modified_annotations_v2.00.csv
! wget https://raw.githubusercontent.com/bhaweshiitk/3.C51_project/main/megares_modified_database_v2.00.fasta

--2022-05-08 23:27:07--  https://raw.githubusercontent.com/bhaweshiitk/3.C51_project/main/megares_modified_annotations_v2.00.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 948917 (927K) [text/plain]
Saving to: ‘megares_modified_annotations_v2.00.csv.1’

megares_modified_an 100%[===================>] 926.68K  --.-KB/s    in 0.03s   

2022-05-08 23:27:07 (29.9 MB/s) - ‘megares_modified_annotations_v2.00.csv.1’ saved [948917/948917]

--2022-05-08 23:27:07--  https://raw.githubusercontent.com/bhaweshiitk/3.C51_project/main/megares_modified_database_v2.00.fasta
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.10

In [7]:
# Process data
train_annotations = pd.read_csv("./megares_modified_annotations_v2.00.csv")
train_annotations.shape

(7868, 5)

In [8]:
train_annotations.head()

,header,type,class,mechanism,group
0,MEG_1|Drugs|Aminoglycosides|Aminoglycoside-res...,Drugs,Aminoglycosides,Aminoglycoside-resistant 16S ribosomal subunit...,A16S
1,MEG_2|Drugs|Aminoglycosides|Aminoglycoside-res...,Drugs,Aminoglycosides,Aminoglycoside-resistant 16S ribosomal subunit...,A16S
2,MEG_3|Drugs|Aminoglycosides|Aminoglycoside-res...,Drugs,Aminoglycosides,Aminoglycoside-resistant 16S ribosomal subunit...,A16S
3,MEG_4|Drugs|Aminoglycosides|Aminoglycoside-res...,Drugs,Aminoglycosides,Aminoglycoside-resistant 16S ribosomal subunit...,A16S
4,MEG_5|Drugs|Aminoglycosides|Aminoglycoside-res...,Drugs,Aminoglycosides,Aminoglycoside-resistant 16S ribosomal subunit...,A16S


In [9]:
train_annotations['Seq_name'] = train_annotations['header'].str.split('|').str[0:2].apply('|'.join)
train_annotations['mechanism'] = train_annotations['header'].str.split('|').str[3]

In [10]:
fasta_sequences = SeqIO.parse(open("./megares_modified_database_v2.00.fasta"),'fasta')
sequence_dict = {}
for fasta in fasta_sequences:
    name, sequence = fasta.id, str(fasta.seq)
    name = "|".join((name.split("|", 2))[0:2])
    sequence_dict[name] = sequence
sequence_df = pd.DataFrame(sequence_dict.items(), columns=['Seq_name', 'Sequence'])
sequence_df.shape

(7868, 2)

In [11]:
sequence_df.head()

,Seq_name,Sequence
0,MEG_1|Drugs,AGAATTTGATCTTGGTTCAGATTGAACGCTGGCGGCGTGGATGAGG...
1,MEG_2|Drugs,AGGTAAGGAGGTGATCCAACCGCAGGTTCCCCTACGGTTACCTTGT...
2,MEG_3|Drugs,TTTTGTTTGGAGAGTTTGATCCTGGCTCAGGACGAACGCTGGCGGC...
3,MEG_4|Drugs,AGAGATTGAACATAAGAGTTTGATCCTGGCTCAGATTGAACGCTGG...
4,MEG_5|Drugs,AGAGTTTGATCCTGGCTCAGGACGAACGCTGGCGGCGTGCTTAACA...


In [12]:
df = pd.merge(train_annotations[['Seq_name', 'type', 'class','mechanism','group']], 
              sequence_df, how = "inner", on = ["Seq_name"])
df.shape

(7868, 6)

In [13]:
df.head()

,Seq_name,type,class,mechanism,group,Sequence
0,MEG_1|Drugs,Drugs,Aminoglycosides,Aminoglycoside-resistant_16S_ribosomal_subunit...,A16S,AGAATTTGATCTTGGTTCAGATTGAACGCTGGCGGCGTGGATGAGG...
1,MEG_2|Drugs,Drugs,Aminoglycosides,Aminoglycoside-resistant_16S_ribosomal_subunit...,A16S,AGGTAAGGAGGTGATCCAACCGCAGGTTCCCCTACGGTTACCTTGT...
2,MEG_3|Drugs,Drugs,Aminoglycosides,Aminoglycoside-resistant_16S_ribosomal_subunit...,A16S,TTTTGTTTGGAGAGTTTGATCCTGGCTCAGGACGAACGCTGGCGGC...
3,MEG_4|Drugs,Drugs,Aminoglycosides,Aminoglycoside-resistant_16S_ribosomal_subunit...,A16S,AGAGATTGAACATAAGAGTTTGATCCTGGCTCAGATTGAACGCTGG...
4,MEG_5|Drugs,Drugs,Aminoglycosides,Aminoglycoside-resistant_16S_ribosomal_subunit...,A16S,AGAGTTTGATCCTGGCTCAGGACGAACGCTGGCGGCGTGCTTAACA...


In [14]:
df["class"].value_counts()

betalactams                                        3911
Aminoglycosides                                     773
Drug and biocide resistance                         363
Glycopeptides                                       299
Fluoroquinolones                                    296
Tetracyclines                                       294
MLS                                                 270
Multi-metal resistance                              226
Phenicol                                            145
Trimethoprim                                        135
Copper resistance                                   117
Multi-drug resistance                               113
Sulfonamides                                        103
Multi-biocide resistance                             98
Lipopeptides                                         95
Rifampin                                             70
Fosfomycin                                           62
Mercury resistance                              

In [15]:
df["mechanism"].value_counts()

Class_A_betalactamases                 1580
Class_C_betalactamases                  950
Class_D_betalactamases                  900
Class_B_betalactamases                  410
Aminoglycoside_N-acetyltransferases     359
                                       ... 
Tetracenomycin_MFS_efflux_pump            1
Tunicamycin_resistance_protein            1
Chromate_ABC_efflux_pump                  1
Biguanide_cation_efflux                   1
Sulfonamide_monooxygenase                 1
Name: mechanism, Length: 219, dtype: int64

In [16]:
label_1 = ['Aminoglycosides', 'betalactams', 'Glycopeptides', 'Tetracyclines', 'MLS', 'Fluoroquinolones', 'Trimethoprim',
           'Phenicol', 'Sulfonamides', 'Sulfonamides', 'Lipopeptides']

In [17]:
df['labels'] = df["class"]
df.loc[~df['labels'].isin(label_1),'labels'] = "other"

In [18]:
df.labels.value_counts()

betalactams         3911
other               1547
Aminoglycosides      773
Glycopeptides        299
Fluoroquinolones     296
Tetracyclines        294
MLS                  270
Phenicol             145
Trimethoprim         135
Sulfonamides         103
Lipopeptides          95
Name: labels, dtype: int64

In [19]:
X = df.Sequence.values
y = df.labels.values
print(X.shape, y.shape)
index_to_remove = []
for (i, seq) in enumerate(X):
  if set(seq.upper())!={"A", "T", "C", "G"}:
    index_to_remove.append(i)
index_to_remove
X = np.delete(X, index_to_remove, axis=0)
y = np.delete(y, index_to_remove, axis=0)
print(X.shape, y.shape)


(7868,) (7868,)
(7819,) (7819,)


In [20]:
code = {'A': 1, 'C': 2, 'G': 3, 'T': 4}
label_code_1 = {"betalactams":0, "Aminoglycosides":1, "Glycopeptides":2, "Fluoroquinolones":3, "Tetracyclines":4, "MLS":5,
                 'Trimethoprim':6,'Phenicol':7,  'Sulfonamides':8, 'Lipopeptides':9,"other":10}


In [21]:
def encode_sklearn(seq):
  code1 = {'A': [1, 0, 0, 0], 'C': [0, 1, 0, 0], 'G': [0, 0, 1, 0], 'T': [0, 0, 0, 1]}

  enc_seq = []
  for x in seq.upper():
    enc_seq += code1[x]
  if len(enc_seq)<2000:
    enc_seq = enc_seq + [0, 0, 0, 0]*(2000-len(enc_seq))
  return enc_seq[0:2000]


In [22]:
X = np.array([encode_sklearn(elem) for elem in X])
y = np.array([label_code_1[label] for label in y])
X.shape, y.shape

((7819, 2000), (7819,))

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) 
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1) 
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1) 

In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV

## Logistic Regression

In [ ]:
%%time
logistic = LogisticRegression(multi_class='multinomial')
distributions = dict(C=[0.5, 1, 1.5], penalty=['l2', 'none'], solver = ['lbfgs', "saga"], class_weight = ['balanced', None])

clf = RandomizedSearchCV(logistic, distributions, cv = 3, verbose=False, n_jobs=-1, random_state = 1)

search = clf.fit(X_train, y_train)


CPU times: user 1min 22s, sys: 622 ms, total: 1min 22s
Wall time: 11min 49s


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


In [ ]:
search.best_params_

{'C': 1, 'class_weight': None, 'penalty': 'l2', 'solver': 'saga'}

In [26]:
acc = []
for i in range(10):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=i) 
  lr = LogisticRegression(multi_class='multinomial', C= 1, class_weight= None, penalty= 'l2', solver= 'saga')
  lr.fit(X_train, y_train)
  y_pred = lr.predict(X_test)
  acc.append((y_pred==y_test).mean())


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter

In [40]:
print("Logistic Regression Accuracy is : %.2f " %(np.array(acc).mean()))
print("Std of Accuracy is : %.2f " %(np.array(acc).std()))

Logistic Regression Accuracy is : 0.89 
Std of Accuracy is : 0.01 


In [37]:
acc

[0.8881074168797954,
 0.8887468030690537,
 0.9021739130434783,
 0.8938618925831202,
 0.8976982097186701,
 0.889386189258312,
 0.889386189258312,
 0.9002557544757033,
 0.887468030690537,
 0.9104859335038363]

## Random Forest

In [ ]:
distributions = {'bootstrap': [True, False],
 'max_depth': [10, 50, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600]}
classifier = RandomForestClassifier(random_state = 42)
clf = RandomizedSearchCV(classifier, distributions, cv = 3, verbose=True, n_jobs=-1, random_state = 1)

search = clf.fit(X_train, y_train)


Fitting 3 folds for each of 10 candidates, totalling 30 fits


In [ ]:
print(search.best_params_)

{'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': 50, 'bootstrap': False}


In [41]:
acc_rf = []
for i in range(10):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=i) 
  rf = RandomForestClassifier(n_estimators=200, min_samples_split= 2, min_samples_leaf= 1, max_features= 'auto', max_depth= 50, 
                              bootstrap = False)
  rf.fit(X_train, y_train)
  y_pred = rf.predict(X_test)
  acc_rf.append((y_pred==y_test).mean())


In [42]:
print("Random Forest Accuracy is : %.2f " %(np.array(acc_rf).mean()))
print("Std of Accuracy is : %.2f " %(np.array(acc_rf).std()))

Random Forest Accuracy is : 0.90 
Std of Accuracy is : 0.01 


## MLP

In [43]:
classifier = MLPClassifier(hidden_layer_sizes=(512, 64), max_iter=300,activation = 'relu',solver='adam',random_state=1)
acc_MLP = []
for i in range(10):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=i)
  classifier.fit(X_train, y_train)

  y_pred = classifier.predict(X_test)
  acc_MLP.append((y_pred==y_test).mean())


In [44]:
print("MLP Accuracy is : %.3f" %(np.array(acc_MLP).mean()))
print("Std of Accuracy is : %.2f " %(np.array(acc_MLP).std()))

MLP Accuracy is : 0.900
Std of Accuracy is : 0.01 


# Mechanism of Antibiotic Resistance Prediction

In [45]:
df["labels_2"] = "other"
df.loc[(df['mechanism'].str.contains("nucleotidyltransferases")),'labels_2'] = "nucleotidyltransferases"
df.loc[(df['mechanism'].str.contains("methyltransferases")),'labels_2'] = "methyltransferases"
df.loc[(df['mechanism'].str.contains("phosphotransferases")),'labels_2'] = "phosphotransferases"
df.loc[(df['mechanism'].str.contains("acetyltransferases")),'labels_2'] = "acetyltransferases"
df.loc[(df['mechanism'].str.contains("Quinolone_resistance_protein_Qnr")),'labels_2'] = "Quinolone_resistance_protein_Qnr"
df.loc[(df['mechanism'].str.contains("Dihydrofolate_reductase")),'labels_2'] = "Dihydrofolate_reductase"
df.loc[(df['mechanism'].str.contains("Tetracycline_resistance_ribosomal_protection_proteins")),
       'labels_2'] = "Tetracycline_resistance_ribosomal_protection_proteins"
df.loc[(df['mechanism'].str.contains("efflux_pump")),'labels_2'] = "efflux_pump"
df.loc[(df['mechanism'].str.contains("Fluoroquinolone-resistant_DNA_topoisomerases")),'labels_2'] = "Fluoroquinolone-resistant_DNA_topoisomerases"
df.loc[(df['mechanism'] == "Class_A_betalactamases"),'labels_2'] = "Class_A_betalactamases"
df.loc[(df['mechanism'] == "Class_B_betalactamases"),'labels_2'] = "Class_B_betalactamases"
df.loc[(df['mechanism'] == "Class_C_betalactamases"),'labels_2'] = "Class_C_betalactamases"
df.loc[(df['mechanism'] == "Class_D_betalactamases"),'labels_2'] = "Class_D_betalactamases"
label_code_2 = {}
labels_2 = ["nucleotidyltransferases", "methyltransferases", "phosphotransferases",
            "acetyltransferases", "Quinolone_resistance_protein_Qnr", "Dihydrofolate_reductase", "efflux_pump",
            "Tetracycline_resistance_ribosomal_protection_proteins", "Fluoroquinolone-resistant_DNA_topoisomerases",
            "Class_A_betalactamases", "Class_B_betalactamases", "Class_C_betalactamases","Class_D_betalactamases", "other"]
for (i, elem) in enumerate(labels_2):
  label_code_2[elem] = i

In [46]:
df.labels_2.value_counts()

other                                                    1642
Class_A_betalactamases                                   1580
Class_C_betalactamases                                    950
Class_D_betalactamases                                    900
efflux_pump                                               826
acetyltransferases                                        445
Class_B_betalactamases                                    410
nucleotidyltransferases                                   296
Quinolone_resistance_protein_Qnr                          173
methyltransferases                                        145
Dihydrofolate_reductase                                   135
Tetracycline_resistance_ribosomal_protection_proteins     134
phosphotransferases                                       122
Fluoroquinolone-resistant_DNA_topoisomerases              110
Name: labels_2, dtype: int64

In [47]:
X = df.Sequence.values
y = df.labels_2.values
print(X.shape, y.shape)
index_to_remove = []
for (i, seq) in enumerate(X):
  if set(seq.upper())!={"A", "T", "C", "G"}:
    index_to_remove.append(i)
index_to_remove
X = np.delete(X, index_to_remove, axis=0)
y = np.delete(y, index_to_remove, axis=0)

print(X.shape,y.shape)

(7868,) (7868,)
(7819,) (7819,)


In [48]:
X = np.array([encode_sklearn(elem) for elem in X])
y = np.array([label_code_2[label] for label in y])
X.shape, y.shape

((7819, 2000), (7819,))

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) 

## Logistic Regression

In [51]:
%%time
logistic = LogisticRegression(multi_class='multinomial')
distributions = dict(C=[0.5, 1, 1.5], penalty=['l2', 'none'], solver = ['lbfgs', "saga"], class_weight = ['balanced', None])

clf = RandomizedSearchCV(logistic, distributions, cv = 3, verbose=False, n_jobs=-1, random_state = 1)

search = clf.fit(X_train, y_train)


In [ ]:
search.best_params_

{'C': 1, 'class_weight': None, 'penalty': 'l2', 'solver': 'saga'}

In [52]:
acc = []
for i in range(10):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=i) 
  lr = LogisticRegression(multi_class='multinomial', C= 1, class_weight= None, penalty= 'l2', solver= 'saga')
  lr.fit(X_train, y_train)
  y_pred = lr.predict(X_test)
  acc.append((y_pred==y_test).mean())

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter

In [53]:
print("Logistic Regression Accuracy is : %.2f " %(np.array(acc).mean()))
print("Std of Accuracy is : %.2f " %(np.array(acc).std()))

Logistic Regression Accuracy is : 0.88 
Std of Accuracy is : 0.01 


## Random Forest

In [54]:
distributions = {'bootstrap': [True, False],
 'max_depth': [10, 50, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600]}
classifier = RandomForestClassifier(random_state = 42)
clf = RandomizedSearchCV(classifier, distributions, cv = 3, verbose=True, n_jobs=-1, random_state = 1)

search = clf.fit(X_train, y_train)
print(search.best_params_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
{'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': 50, 'bootstrap': False}


In [55]:
acc_rf = []
for i in range(10):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=i) 
  rf = RandomForestClassifier(n_estimators=200, min_samples_split= 2, min_samples_leaf= 1, max_features= 'auto', max_depth= 50, 
                              bootstrap = False)
  rf.fit(X_train, y_train)
  y_pred = rf.predict(X_test)
  acc_rf.append((y_pred==y_test).mean())


In [56]:
print("Random Forest Accuracy is : %.2f " %(np.array(acc_rf).mean()))
print("Std of Accuracy is : %.2f " %(np.array(acc_rf).std()))

Random Forest Accuracy is : 0.88 
Std of Accuracy is : 0.01 


## MLP

In [57]:
classifier = MLPClassifier(hidden_layer_sizes=(512, 64), max_iter=300,activation = 'relu',solver='adam',random_state=1)
acc_MLP = []
for i in range(10):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=i)
  classifier.fit(X_train, y_train)

  y_pred = classifier.predict(X_test)
  acc_MLP.append((y_pred==y_test).mean())


In [58]:
print("MLP Accuracy is : %.3f" %(np.array(acc_MLP).mean()))
print("Std of Accuracy is : %.2f " %(np.array(acc_MLP).std()))

MLP Accuracy is : 0.884
Std of Accuracy is : 0.01 
